### **Setup**

In [ ]:
! pip install --quiet datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 25.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is inc

### **Load Dataset**

In [ ]:
from datasets import load_dataset

ner_dataset = load_dataset("rasyosef/amharic-named-entity-recognition", split="train")
ner_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


Generating train split:   0%|          | 0/3465 [00:00<?, ? examples/s]

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 3465
})

In [ ]:
ner_dataset.features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-TIME', 'I-TIME', 'B-TTL', 'I-TTL'], id=None), length=-1, id=None)

In [ ]:
categories = ner_dataset.features["ner_tags"].feature.names

label2id = {
  label: i for i, label in enumerate(categories)
}

id2label = {
  v: k for k, v in label2id.items()
}

print(id2label)
print(label2id)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-TIME', 8: 'I-TIME', 9: 'B-TTL', 10: 'I-TTL'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-TIME': 7, 'I-TIME': 8, 'B-TTL': 9, 'I-TTL': 10}


### **Processing the data**

In [ ]:
# Load Tokenizer
from transformers import AutoTokenizer

checkpoint = "rasyosef/bert-medium-amharic"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/274k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/725k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast

True

In [ ]:
# Tokenize the pretokenized input by adding is_split_into_words=True
inputs = tokenizer(ner_dataset[0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['[CLS]', 'ኢዴፓ', 'በየ', '##ክልሉ', 'በሚ', '##ንቀሳቀስ', '##በት', 'ጊዜ', 'ሁሉ', 'የሀገሪቱን', 'አጠቃላይ', 'ሕግ', '##እን', '##ዲሁም', 'የአካባቢውን', 'ባህልና', 'ቋንቋ', 'አክብሮ', 'በአካባቢው', 'የሚገኙ', 'የፖለቲካ', 'ድርጅቶችን', '##ም', 'አክብሮ', '##ና', 'መብታቸውን', 'ጠብቆ', 'በጨ', '##ዋ', '##ነት', 'ያስተም', '##ራል', '፣', 'ይ', '##ማራ', '##ል', '"', 'ብለዋል', '።', '[SEP]']


In [ ]:
print(inputs.word_ids())

[None, 0, 1, 1, 2, 2, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15, 15, 16, 16, 17, 18, 19, 19, 19, 20, 20, 21, 22, 22, 22, 22, 23, 24, None]


In [ ]:
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id != current_word: # start of new word
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None: # special token
      new_labels.append(-100)
    else:
      # same word as pervious token
      label = labels[word_id]
      # If label is B-XXX change it to I-XXX
      if id2label[label].startswith("B-"):
        label = label2id["I-" + id2label[label][2:]]
      new_labels.append(label)
  return new_labels


In [ ]:
labels = ner_dataset[2]["ner_tags"]
inputs = tokenizer(ner_dataset[2]["tokens"], is_split_into_words=True)
word_ids = inputs.word_ids()
print(inputs.tokens())
print(labels)
print(align_labels_with_tokens(labels, word_ids))

['[CLS]', 'በባህር', 'ዳር', 'ዩኒቨርስቲ', '##ና', 'በጅማ', 'መምህራን', 'ኮሌጅ', 'ለነ', '##ባር', 'ተማሪዎች', 'የምግብ', '##ና', 'የመ', '##ኝታ', 'አገልግሎት', 'ሊያ', '##ቆም', 'እንደሚችል', 'ባለፈው', 'ዓመት', 'የተሰጠው', 'ማሳሰቢያ', 'ከዚህ', 'ዓመት', 'ጀምሮ', 'ተግባራዊ', 'ይሆናል', 'በመ', '##ባሉ', 'ተማሪዎች', 'ከፍተኛ', 'ስጋት', '##ና', 'ጭንቀት', 'ላይ', 'መውደ', '##ቃቸውን', 'ተጠቆመ', '።', '[SEP]']
[3, 4, 4, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 3, 4, 4, 4, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(
      examples['tokens'], truncation=True, is_split_into_words=True
  )
  all_labels = examples['ner_tags']
  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels
  return tokenized_inputs

In [ ]:
tokenized_datasets = ner_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=ner_dataset.column_names,
)
tokenized_datasets

Map:   0%|          | 0/3465 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3465
})

In [ ]:
preprocessed_datasets = tokenized_datasets.train_test_split(test_size=0.2, seed=16)
preprocessed_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2772
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 693
    })
})

### **Finetuning** `bert-medium-amharic`

In [ ]:
# Data Collator
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([preprocessed_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    5,    6,    6,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

### **Metrics**

In [ ]:
!pip install --quiet seqeval

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
labels = preprocessed_datasets["train"][3]["labels"]
print(labels)
labels = [categories[i] for i in labels[1:-1]]
print(labels), labels[19]

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


(None, 'O')

In [ ]:
predictions = labels.copy()
predictions[19] = "O"
metric.compute(predictions=[predictions], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[categories[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [categories[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

### **Defining the Model**

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at rasyosef/bert-medium-amharic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Number of labels
model.config.num_labels

11

### **Fine-tuning the model**

In [ ]:
from transformers import TrainingArguments

batch_size = 32
args = TrainingArguments(
    "bert-medium-amharic-finetuned-ner",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=4e-5,
    #lr_scheduler_type="linear",
    num_train_epochs=12,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    seed=42
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=preprocessed_datasets["train"],
    eval_dataset=preprocessed_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.548800,0.223003,0.438746,0.345679,0.386692,0.938732
2,0.192300,0.150144,0.559524,0.527497,0.543039,0.954184
3,0.130800,0.129360,0.552707,0.653199,0.598765,0.956874
4,0.102500,0.129382,0.592809,0.684624,0.635417,0.957451
5,0.083700,0.129855,0.627392,0.699214,0.661359,0.960910
6,0.071800,0.125908,0.608571,0.717172,0.658423,0.959988
7,0.061200,0.132793,0.629593,0.711560,0.668072,0.961218
8,0.053100,0.134769,0.638086,0.718294,0.675818,0.962179
9,0.048600,0.140947,0.637341,0.731762,0.681296,0.960833
10,0.044600,0.143455,0.643781,0.726150,0.682489,0.961295


TrainOutput(global_step=1044, training_loss=0.11830754869285671, metrics={'train_runtime': 227.072, 'train_samples_per_second': 146.491, 'train_steps_per_second': 4.598, 'total_flos': 487981225241664.0, 'train_loss': 0.11830754869285671, 'epoch': 12.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.1450619101524353,
 'eval_precision': 0.6445544554455446,
 'eval_recall': 0.7306397306397306,
 'eval_f1': 0.6849026827985272,
 'eval_accuracy': 0.9617557750701464,
 'eval_runtime': 2.3149,
 'eval_samples_per_second': 299.359,
 'eval_steps_per_second': 9.503,
 'epoch': 12.0}

### **Testing**

In [ ]:
from transformers import pipeline

ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:
ind = 3
text = " ".join(ner_dataset[ind]["tokens"])
print(text)
print(" ".join([str(i) for i in ner_dataset[ind]["ner_tags"]]))
ner_pipe(text)

የሻዕቢያው መሪ ለዚህ ማስፈራሪያቸው እንደ አብነት የተጠቀሙበት ኦጋዴንን ሲሆን የኢሕአዴግ መንግሥት ከቅኝ ግዛት ውሎቹ ላፈንግጥ ቢል ያንን ግዛት ለዘለዓለሙ ሊያጣው እንደሚችል ግልጽ ሊሆንለት ይገባል ሲሉ አስጠንቅቀዋል ።
3 0 0 0 0 0 0 5 0 3 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


[{'entity_group': 'ORG',
  'score': 0.9508012,
  'word': 'የሻዕቢያው',
  'start': 0,
  'end': 6},
 {'entity_group': 'LOC',
  'score': 0.94448346,
  'word': 'ኦጋዴንን',
  'start': 40,
  'end': 45},
 {'entity_group': 'ORG',
  'score': 0.96137446,
  'word': 'የኢሕአዴግ መንግሥት',
  'start': 50,
  'end': 62}]

In [ ]:
ind = 12
text = " ".join(ner_dataset[ind]["tokens"])
print(text)
print(" ".join([str(i) for i in ner_dataset[ind]["ner_tags"]]))
ner_pipe(text)

በናዝሬት አጠቃላይ ዕድሮች የመሰብሰቢያ አዳራሽ በተካሄደው በዚሁ ስብሰባ ላይ በአዳራሽ ውስጥ በመቀመጥና በመቆም ፣ ከውጭም በመስኮት ቁጥሩ እስከ 1400 የሆነ ሕዝብ መገኘቱን የገለጡት አቶ ልደቱ መታፈስ ሊኖር ይችላል" የሚል ማስፈራሪያ በከተማው ተሠራጭቶ የነበረ ቢሆንም ቁጥራቸው የበዛ ወጣቶች ጐልማሶችና አዛውንቶች በስብሰባው ላይ መካፈላቸውን ፣ ብዙውም በአዳራሽ ጥበት ምክንያትም መመለሱን አመልክተዋል ።
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 9 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


[{'entity_group': 'LOC',
  'score': 0.69236934,
  'word': 'በናዝሬት',
  'start': 0,
  'end': 5},
 {'entity_group': 'TTL',
  'score': 0.9899262,
  'word': 'አቶ',
  'start': 117,
  'end': 119},
 {'entity_group': 'PER',
  'score': 0.9940798,
  'word': 'ልደቱ',
  'start': 120,
  'end': 123}]